In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

model = ChatOpenAI(model="gpt-4o")

# Create specialized agents

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)



In [4]:
workflow = create_supervisor(
    [research_agent, math_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )
)

In [5]:
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what's the combined headcount of the FAANG companies in 2024?"
        }
    ]
})

In [6]:
result

{'messages': [HumanMessage(content="what's the combined headcount of the FAANG companies in 2024?", additional_kwargs={}, response_metadata={}, id='647fb354-aebb-42f9-bb55-434fdf284172'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LlzrTtOdGsCa2WRMA5n89Pii', 'function': {'arguments': '{}', 'name': 'transfer_to_research_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 104, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ff092ab25e', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-c7844f24-b80f-4372-8ebf-3b6e7f7bb66a-0', tool_calls=[{'name': 'transfer_to_research_expert', 'args': {}, 'id': 'call_LlzrTtOdGsCa2WRMA5n89Pii'

### Supervisor Agent with low level agent

In [7]:
from typing import Literal


from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt import ToolNode

# --- Weather tool ---
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."

# We'll create a model and bind the tool so the LLM knows it can call `get_weather`.
tools = [get_weather]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

# --- Existing agent workflow definition ---
def call_model(state: MessagesState):
    """Call the LLM with the conversation so far."""
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", END]:
    """If there's a tool call requested, go to 'tools', else end."""
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

weather_workflow = StateGraph(MessagesState)

tool_node = ToolNode(tools)

weather_workflow.add_node("agent", call_model)
weather_workflow.add_node("tools", tool_node)

weather_workflow.add_edge(START, "agent")
weather_workflow.add_conditional_edges("agent", should_continue)
weather_workflow.add_edge("tools", "agent")

weather_agent_graph =  weather_workflow.compile(name="weather_agent")

In [11]:
from langgraph_supervisor import create_supervisor

supervisor_workflow = create_supervisor(
    agents=[weather_agent_graph],
    model=model,
    prompt=(
        "You are a supervisor managing a weather agent. "
        "For any weather-related question, call the 'weather_agent' to handle it."
    ),
    output_mode="last_message",
    #output_mode="full_history",
    supervisor_name="supervisor_agent",
)

supervisor_app = supervisor_workflow.compile()

In [12]:
supervisor_app.invoke(
    {"messages": [HumanMessage(content="Hello there, how are you?")]}
)

{'messages': [HumanMessage(content='Hello there, how are you?', additional_kwargs={}, response_metadata={}, id='2d7b2062-da5e-44bf-9038-cfda9aba1ba8'),
  AIMessage(content="I'm just a program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 71, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'stop', 'logprobs': None}, name='supervisor_agent', id='run-f844fa58-516e-4cd8-a2be-5e80aedb5fde-0', usage_metadata={'input_tokens': 71, 'output_tokens': 29, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

In [13]:
supervisor_app.invoke(
    {"messages": [HumanMessage(content="How is the weather in Munich?")]}
)


{'messages': [HumanMessage(content='How is the weather in Munich?', additional_kwargs={}, response_metadata={}, id='7b6e7f69-19f3-4af5-9327-66c0478cfe68'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pU8pdeSkHkerBjNZ9KrzGTe6', 'function': {'arguments': '{}', 'name': 'transfer_to_weather_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 71, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor_agent', id='run-f01e26aa-d1ad-420a-ae5f-8fca1376adaa-0', tool_calls=[{'name': 'transfer_to_weather_agent', 'args': {}, 'id': 'call_pU8pdeSkHkerBjNZ9KrzGTe6', 'type': 'tool_call'}], us